In [45]:
import pandas as pd
import os
from pyDOE import *
from scipy.io import netcdf as nc
import xarray as xr
import copy
import netCDF4

## Download latest version of params file from google drive
* requires 'publishing' the google drive spreadsheet
* file > publish to web
* then it can be set up to continuously publish the spreadsheet to a stable url (with some latency, maybe 1-2 minutes)
* note that the first tab must be the sheet where the relevant information is located

In [2]:
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
#cmd = 'curl '+data_url+' > params.csv'
cmd = 'curl -L '+data_url+' > params.csv' # need to add -L option to force redirects
os.system(cmd)

0

## Defining a class for organizing parameter information

In [13]:
class ParamInfo(object):
    """
    Stores parameter information
    """
    
    def __init__(self, name, loc, minval=None, maxval=None, defval=None):
        self._name = name # parameter name
        self._min = minval # minimum value
        self._max = maxval # maximum value
        self._default = defval # default value
        self._value = None # actual value to be used in a given ensemble member
        self._location = loc # location of parameter (params file or namelist)

    @property
    def name(self):
        return self._name

    @property
    def min(self):
        return self._min

    @property
    def max(self):
        return self._max
    
    @property
    def default(self):
        return self._default
    
    @property
    def value(self):
        return self._value
    
    @property
    def location(self):
        return self._location
    
    @name.setter
    def name(self, new_name):
        self._name = new_name
   
    @min.setter
    def min(self, new_min):
        self._min = new_min
        
    @max.setter
    def max(self, new_max):
        self._max = new_max
        
    @default.setter
    def default(self, new_def):
        self._default = new_def
        
    @value.setter
    def value(self, new_val):
        self._value = new_val
            
    def __repr__(self):
        return "%s:\n\tloc = %s\n\tdefault = %s\n\tmin = %s\n\tmax = %s\n\tvalue = %s" % (self.name, self.location, self.default, self.min, self.max, self.value)

In [14]:
# TO DO: add these unit tests to ParamVal class itself
# Check every time the code updates, or adding new functionality

# testing out the class/dictionary functionality
test_dict = {"P1": ParamInfo("P1", minval=0.0, maxval=1.0, defval=2.0, loc='N'),
             "P2": ParamInfo("P2", minval=[0,0,0,0,0], maxval=[100,100,100,100,100], defval=[0,1,2,3,4], loc='P'),
             "P3": ParamInfo("P3", minval="min", maxval="max", defval="value", loc='N'),
             "P4": ParamInfo("P4", loc='P')
            }

# example of adding a new parameter
test_dict["new_param"] = ParamInfo("new_param", 'N')

# example of setting the max value
test_dict["P4"].max = 200

# example of setting the value for a given ensemble member
test_dict["new_param"].value = 100

# look at the test dictionary
for key in test_dict:
    print(test_dict[key])

P1:
	loc = N
	default = 2.0
	min = 0.0
	max = 1.0
	value = None
P2:
	loc = P
	default = [0, 1, 2, 3, 4]
	min = [0, 0, 0, 0, 0]
	max = [100, 100, 100, 100, 100]
	value = None
P3:
	loc = N
	default = value
	min = min
	max = max
	value = None
P4:
	loc = P
	default = None
	min = None
	max = 200
	value = None
new_param:
	loc = N
	default = None
	min = None
	max = None
	value = 100


## Defining a class for organizing ensemble members

In [106]:
class Member(object):
    """
    Stores and works with a bunch of ParamInfos
    """

    def __init__(self, name, paraminfo):
        self._name = name
        self._paraminfo = paraminfo
        
    @property
    def name(self):
        return self._name
    
    @property
    def paraminfo(self):
        return self._paraminfo
    
    @name.setter
    def name(self, new_name):
        self._name = new_name
    
    def get_names(self):
        """
        Returns a list of parameter names.
        """
        
        names = []
        for param in self._paraminfo:
            names.append(self._paraminfo[param].name)
        return names
                 
    def write(self, sampling_protocol):
        """
        Writes files to disk for each member: param netcdf, namelist mods txt.
        """
        
        # generate file names based on member name
        i = int(self._name)
        self._pftfile = "../paramfiles/"+sampling_protocol+str(i+1).zfill(4)+".nc"
        self._nlfile = "../namelist_mods/"+sampling_protocol+str(i+1).zfill(4)+".txt" 
        
        # assign the basepftfile (this also happens in Ensemble class)
        self._basepftfile = "../basecase/clm5_params.c200519.nc"
        
        # create the pftfile as a copy of the basepftfile
        cmd = 'cp '+self._basepftfile+' '+self._pftfile
        os.system(cmd)
        print('working on '+self._pftfile)
        
        # create the nlfile
        with open(self._nlfile,"w") as file:
            output = "! user_nl_clm namelist options written by generate_params:\n"
            file.write(output)
        print('working on '+self._nlfile)
            
        # read in the pftfile using netCDF4 package
        dset = netCDF4.Dataset(self._pftfile,'r+')
        
        # modify the pft/nl files
        if sampling_protocol == "OAAT":
            for paramname in self.get_names():
                # for OAAT, only modify if value is not 'None'
                if self._paraminfo[paramname].value is not None:
                    # params file
                    if self._paraminfo[paramname].location == "P":
                        print(paramname+' modified in pftfile')
                        var = dset[paramname]
                        dset[paramname][:] = self._paraminfo[paramname].value
                    # namelist
                    elif self._paraminfo[paramname].location == "N":
                        print(paramname+' modified in nlfile')
                        with open(self._nlfile,"a") as file: # key is using "a" for append option
                            output = "%s=%s\n" % (paramname, self._paraminfo[paramname].value) #round??
                            file.write(output)

        # TO DO: LHC code for writing files
        elif sampling_protocol == "LHC":
            pass
        
        
        # need to "close" netcdf file to write out
        dset.close() 
        
        
    def __repr__(self):
        return "Ensemble member %s" % (self._name, )

In [16]:
# TO DO: add these unit tests to Member class itself
# Check every time the code updates, or adding new functionality

# testing out the class/dictionary functionality
member_test_dict = {"M1": Member("M1", paraminfo=test_dict),
             "M2": Member("M2", paraminfo=test_dict),
             "M3": Member("M3", paraminfo=None),
             "M4": Member("M4", paraminfo=None)
            }

# example of adding a new member
member_test_dict["new_member"] = Member("new_member", paraminfo=test_dict)

# example of setting the name
member_test_dict["new_member"].name = "M5"

# look at the test dictionary
for key in member_test_dict:
    print(member_test_dict[key])

Ensemble member M1
Ensemble member M2
Ensemble member M3
Ensemble member M4
Ensemble member M5


In [54]:
# look at a member's paraminfo in the test dictionary
member_test_dict['M1'].paraminfo
# look at a member's paraminfo for a specific parameter
member_test_dict['M1'].paraminfo['P1']
# get the list of parameter names
member_test_dict['M1'].get_names()

['P1', 'P2', 'P3', 'P4', 'new_param']

## Define a class for organizing the ensemble

In [96]:
class Ensemble(object):
    """
    Stores and works with a bunch of Members
    """
    
    # assign the basepftfile
    _basepftfile = "../basecase/clm5_params.c200519.nc"
    
    # NOTE: here using an example lnd_in file to pull in default namelist values
    # Could also parse the namelist defaults file, see: https://github.com/ESCOMP/CTSM/blob/e2b9745d81ed5cb7cd7f5d6098edf506a4956335/bld/namelist_files/namelist_defaults_ctsm.xml
    _thedir = '/glade/work/djk2120/ctsm_hardcode_co/cime/scripts/clm50c6_ctsmhardcodep_2deg_GSWP3V1_Sparse250_2000/CaseDocs/'
    _thefil = 'lnd_in'
    _lndin = _thedir+_thefil
    
    def __init__(self, csvfile, sampling_protocol):
        self._sampling_protocol = sampling_protocol
        
        # Read in csv data, filtering by the "include" column
        #data     = pd.read_csv('params.csv')
        data = pd.read_csv(csvfile,header=0,skiprows=[1]) # modify read_csv to account for header spanning 2 rows
        included = data['include'] == 1
        params_full = data.loc[included,['name','location','min','max','pft_mins','pft_maxs']]

        # reset indexing and get rid of excel row number
        params = params_full.reset_index(drop=True)
        
        # get number of parameters
        self._nparam = len(params['name'])
        
        # declare a dictionary to store parameter information
        self._params_dict = {}
        
        # read in default file
        def_params = xr.open_dataset(self._basepftfile)
        
        # reading in the default values
        # loop over parameters grabbing name and location
        for name,loc in zip(params['name'],params['location']):      
            
            # select parameters located in the params file only
            if loc=='P':
                # getting parameter dims (i.e., checking for segment variation)
                dims = len(def_params[name].values.shape)
                if dims<2:
                    # no segment variation
                    x = def_params[name].values
                else:
                    # segment variation: kmax,ck,psi50,rootprof_beta
                    # assumes same values applied across segments
                    # TO DO: check this assumption, appears not true for rootprof_beta
                    x = def_params[name][0,:].values
                self._params_dict[name] = ParamInfo(name, defval=x, loc='P')
            
            # select namelist parameters
            elif loc=='N':
                # build a command to search lndin for the parameter by name and put output in a tmp file
                cmd = 'grep '+name+' '+self._lndin+' > tmp.txt'
                ret = os.system(cmd)
                # checking for nonzero return code (exit status?), meaning parameter is not found
                if ret != 0:
                    # TO DO: will need to address these special cases somehow...
                    print(name+' not found')
                else:
                    f = open('tmp.txt', 'r')
                    # parse the value from the parameter name
                    tmp = f.read().split()[2]
                    f.close()
                    # cases where scientific notation(?) is specified by a "d"
                    # TO DO: there may be other special cases as well (scientific notation as an "e"?)
                    if 'd' in tmp:
                        tmp = tmp.split('d')
                        x = float(tmp[0])*10**float(tmp[1])
                    else:
                        x = float(tmp)
                    self._params_dict[name] = ParamInfo(name, defval=x, loc='N')
        
        # assigning min and max values
        if sampling_protocol == 'OAAT':
            for i in range(self._nparam):
                # check for pft variation
                if params['min'].values[i]=='pft':
                    self._params_dict[params['name'].values[i]].min = np.fromstring(params['pft_mins'][i],dtype='float',sep=',')
                    self._params_dict[params['name'].values[i]].max = np.fromstring(params['pft_maxs'][i],dtype='float',sep=',')
                # check for "XXpercent" perturb from default
                elif "percent" in params['min'].values[i]:
                    percent_perturb = float(params['min'].values[i].split("percent")[0])
                    percent_min_values = self._params_dict[params['name'].values[i]].default*(1 - percent_perturb/100)
                    percent_max_values = self._params_dict[params['name'].values[i]].default*(1 + percent_perturb/100)            
                    self._params_dict[params['name'].values[i]].min = percent_min_values
                    self._params_dict[params['name'].values[i]].max = percent_max_values        
                else:
                    # assign min/max values directly
                    self._params_dict[params['name'].values[i]].min = params['min'].values[i]
                    self._params_dict[params['name'].values[i]].max = params['max'].values[i]
        elif sampling_protocol == 'LHC':
            # TO DO: assign LHC min/maxes
            pass
        
        # declare a dictionary to store member information
        self._members = {}
        
        # set the number of ensemble members
        if sampling_protocol == 'OAAT':
            # number of samples is twice the number of parameters (min and max perturbations)
            nsamp = 2*self._nparam
        elif sampling_protocol == 'LHC':
            # define sample size for LHC (user-specified)
            nsamp = 10
        
        # create the members
        # need to "deepcopy" the dictionary for each member so they can be modified independently
        for i in range(nsamp):
            self._members[i] = Member(str(i), copy.deepcopy(self._params_dict))
        
        # loop over members and calculate parameter values for each member                
        # TO DO: OAAT ONLY - NEED TO ADD LHC CODE
        doneparams = []
        for member in self._members:
            for paramname in self._members[member].get_names():
                # check if this parameter has already been assigned
                if paramname in doneparams:
                    continue
                # check if this is a min or max perturbation
                if int(self._members[member].name)%2 == 0:
                    # min values
                    self._members[member].paraminfo[paramname].value = self._members[member].paraminfo[paramname].min
                    break
                else:
                    # max values
                    self._members[member].paraminfo[paramname].value = self._members[member].paraminfo[paramname].max
                    # parameter is "done" after creating min/max ensemble members in sequence
                    doneparams.append(paramname)
                    break

        
    @property
    def params(self):
        return self._params_dict
    
    @property
    def members(self):
        return self._members
    
    def output_files(self):
        """
        Loop over members in the ensemble and call the write function.
        """
        
        for member in self._members:
            self._members[member].write(self._sampling_protocol)
    
    def save_psets(self):
        """
        Save the parameter values for the ensemble.
        """
        pass

In [107]:
# Using this Ensemble class with the input csv file
newEns = Ensemble('params.csv', 'OAAT')

# get the params_dict - but which member do the values correspond to?
#newEns.params

# example of how to print member/param info
#for member in newEns.members:
#    print(newEns.members[member])
#    print(newEns.members[member].paraminfo)
    
# example of how to print paraminfo for a given member
#newEns.members[7].paraminfo

# example of how to print paraminfo of a specific parameter for a given member
#newEns.members[0].paraminfo['fff']

# example of how to print paraminfo of a specific parameter for all members
#for member in newEns.members:
#    print(newEns.members[member].paraminfo['fff'])

# example of how to print the "value" info of a specific parameter for all members
#for member in newEns.members:
#    print(newEns.members[member].paraminfo['fff'].value)

# get parameter names for a specify ensemble member
#newEns.members[0].get_names()

In [108]:
# Testing writing out files
newEns.output_files()

working on ../paramfiles/OAAT0001.nc
working on ../namelist_mods/OAAT0001.txt
dleaf modified in pftfile
working on ../paramfiles/OAAT0002.nc
working on ../namelist_mods/OAAT0002.txt
dleaf modified in pftfile
working on ../paramfiles/OAAT0003.nc
working on ../namelist_mods/OAAT0003.txt
baseflow_scalar modified in nlfile
working on ../paramfiles/OAAT0004.nc
working on ../namelist_mods/OAAT0004.txt
baseflow_scalar modified in nlfile
working on ../paramfiles/OAAT0005.nc
working on ../namelist_mods/OAAT0005.txt
maximum_leaf_wetted_fraction modified in nlfile
working on ../paramfiles/OAAT0006.nc
working on ../namelist_mods/OAAT0006.txt
maximum_leaf_wetted_fraction modified in nlfile
working on ../paramfiles/OAAT0007.nc
working on ../namelist_mods/OAAT0007.txt
fff modified in pftfile
working on ../paramfiles/OAAT0008.nc
working on ../namelist_mods/OAAT0008.txt
fff modified in pftfile
working on ../paramfiles/OAAT0009.nc
working on ../namelist_mods/OAAT0009.txt
medlynslope modified in pftfile


### Some notes on the 12-parameter test ensemble:
* leafcn has a non-vegetated default value of 1, so the min/max arrays change this value when a percent perturbation is applied
* don't currently have checks to see if min or max are equivalent to default value (don't change params files / don't make a namelist mod?) - precision/round-off errors could be important here

## Generate parameter sampling -- integrate this code above for LHC option
 * ### careful, each time you run LHC you get a new random draw

In [ ]:
if sampling_protocol == 'LHC':
    # define sample size (number of ensemble members)
    nsamp = 10

    # Generate the latin hypercube sample
    lhd = lhs(nparam, samples=int(nsamp))
    # lhd is a 2D array indexed by ensemble member x parameter
    
    # figure out how many pft-dependent params there are in this sample
    npftparam = sum(params['min']=='pft')
    
    if npftparam>0:
        # get dataframe index of first pft param
        pftfirstind = params.index[params['min']=='pft'][0]
        
        # get number of pfts
        npft = len(np.fromstring(params['pft_mins'][pftfirstind],dtype='float',sep=','))
        
        # set up numpy array to store pft-specific values
        pft_array = np.nan*np.ones([npftparam,npft,nsamp])
        
        for j in range(npftparam):
            # get the index for the current pft param
            pftind = params.index[params['min']=='pft'][j]
            
            # get min values
            min_pft_array = np.fromstring(params['pft_mins'][pftind],dtype='float',sep=',')
            # max values
            max_pft_array = np.fromstring(params['pft_maxs'][pftind],dtype='float',sep=',')
            
            # loop over samples and calculate parameter values for each pft
            for i in range(nsamp):
                pft_array[j,:,i] = (max_pft_array - min_pft_array)*lhd[i,pftind] + min_pft_array
                # can't store pft_array as a pandas dataframe because it's 3D
                # unless there is some alternate way to store this data?
    
    # initialize min/max arrays - for params without pft-variation
    min_array = np.nan*np.ones(nparam)
    max_array = np.nan*np.ones(nparam)
    
    # generate arrays with min and max values
    for i in range(nparam):
        if params['min'].values[i]=='pft':
            # TO DO: what's a good placeholder, to denote need to reference pft_array?
            # numpy doesn't like assigning a string to an existing array of floats
            # for now, just print a message
            print('skipping '+params['name'].values[i]+'...this parameter varies with PFT')
            
            # Numpy doesn't like assigning an array to a single index in an existing array
            # The problem is still that I'm declaring min_array before trying to assign values
            # If I could build it all at once, numpy would allow for nested arrays
            #min_array[i] = np.fromstring(params['pft_mins'].values[i],dtype='float',sep=',')
            #max_array[i] = np.fromstring(params['pft_maxs'].values[i],dtype='float',sep=',')
        else:
            # assign min/max values
            min_array[i] = float(params['min'].values[i])
            max_array[i] = float(params['max'].values[i])
            
    # calculate parameter values; skip pft params (NaNs in min/max arrays)
    param_array = (max_array - min_array)*lhd + min_array

# store psets in a pandas dataframe
#psets = pd.DataFrame(data=param_array, index=None, columns=params['name'])
#psets

## Save off the parameter sets -- need to intergrate into "save_psets" function above

In [54]:
# create a name for this particular ensemble
ensemble_name = "test0001"
# build the file name with the prefix (ensemble type)
psetsfile = "../parameter_sets/"+prefix+"_"+ensemble_name+".csv"
#print(psetsfile)

# first, save the psets dataframe to csv
psets.to_csv(psetsfile)

# second, save the pft array (if applicable)
pftarrayfile = "../parameter_sets/"+prefix+"_"+ensemble_name+"_pftvals"
#print(pftarrayfile)
# save as a numpy array (for now, easiest solution for 3D array?)
np.save(pftarrayfile, pft_array)
# example of how to load it back in
#test = np.load(pftarrayfile+".npy")